In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("test-twist.geo")

Info    : Reading 'test-twist.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.000329045s, CPU 0.000318s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0206254s, CPU 0.020592s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.00495319s, CPU 0.004951s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.00997796s, CPU 0.009012s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.0

In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 45)


LowLevelFEM.Problem("test-twist", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 436)

In [4]:
supp = FEM.displacementConstraint("supp", ux=0, uz=0)
ff(x, y, z) = √(y^2 + z^2)
load = FEM.load("load", fz=ff)

("load", 0, 0, Main.ff)

In [5]:
n1x(x, y, z) = y
n1y(x, y, z) = z
n2x(x, y, z) = -z
n2y = n1x
cs = FEM.CoordinateSystem([n1x, n1y, 0], [n2x, n2y, 0])
Q = FEM.rotateNodes(problem, "body", cs)

LowLevelFEM.Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 4, 5, 6  …  1303, 1304, 1303, 1304, 1305, 1306, 1307, 1306, 1307, 1308], [1, 1, 2, 2, 3, 4, 4, 5, 5, 6  …  1303, 1303, 1304, 1304, 1305, 1306, 1306, 1307, 1307, 1308], [2.4492935982947064e-16, 1.0, -1.0, 2.4492935982947064e-16, 1.0, 2.4492935982947064e-16, 1.0, -1.0, 2.4492935982947064e-16, 1.0  …  -0.9693023760001109, -0.24587172240934843, 0.24587172240934843, -0.9693023760001109, 1.0, -0.4133062019964547, -0.9105921059350699, 0.9105921059350699, -0.4133062019964547, 0.9999999999999998], 1308, 1308), 436, 3)

In [6]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f3D)

In [7]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [8]:
q = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [-0.053624464615837406; 0.023824875921565375; … ; 0.023112857107774356; 0.1965274282656064;;], [0.0], Int64[], 1, :u3D)

In [9]:
FEM.showDoFResults(problem, Q * q, :uvec, visible=true)

MethodError: MethodError: *(::LowLevelFEM.Transformation, ::LowLevelFEM.VectorField) is ambiguous.

Candidates:
  *(A::LowLevelFEM.Transformation, B)
    @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/general.jl:175
  *(b, A::LowLevelFEM.VectorField)
    @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/general.jl:1277

Possible fix, define
  *(::LowLevelFEM.Transformation, ::LowLevelFEM.VectorField)


In [10]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [11]:
gmsh.finalize()